In [4]:
import os
# import args
import openai
# #Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
import json


beth_ast = "/home/yuj49/ConText/i2b2_ast/beth_ast"
beth_txt = "/home/yuj49/ConText/i2b2_ast/beth_txt"

tree_of_thought = {}

chain_of_thought = {}


def i2b2_annotation():
    annotations = []
    count = 0
    for ast_name in os.listdir(beth_ast):
        ast_path = os.path.join(beth_ast, ast_name)
        file_name = ast_name.split(".ast")[0]
        txt_name = file_name+".txt"
        txt_path = os.path.join(beth_txt, txt_name)
        print(ast_name)
        with open(ast_path, "r") as ast_f, open(txt_path, "r") as txt_f:
            txt_list = [i for i in txt_f.readlines()]
            # idx_count = 0
            for line in ast_f.readlines():
                
                # if idx_count > 20:
                #     break
                parts = line.strip().split('||')
                entity = parts[0].split('=')[1].split('"')[1]
                start2end = parts[0].split('=')[1].split('"')[2]
                row_idx = int(start2end.split(":")[0])
                label = parts[2].split('=')[1].strip('"') 
                if label == "absent":
                    label = "Negated"
                if label == "present":
                    label = "Positive"

                item = {"entity":entity, 
                        "row_idx": row_idx, 
                        "label": label,
                        "text": txt_list[row_idx - 1],
                        "file_name": ast_name.split(".ast")[0]}
                print("text",txt_list[row_idx - 1])
                # res = chatgpt_tookit(entity, txt_list[row_idx - 1]) # 287 time call api
                # print(res)
                # item["res"] = res
                # idx_count+=1
                annotations.append(item)
                
    with open("/home/yuj49/ConText/ConText_LLM/chatgpt_i2b2.json", "w") as outfile:
        json.dump(annotations, outfile)
i2b2_annotation()

record-22.ast
text Patient recorded as having No Known Allergies to Drugs

record-141.ast
text The cardiac monitor showed occasional irregularities in a PACED rhythm .

text Pt. was initially admitted to CCU on 2017-09-03 , transferred to Pendleton on 2017-09-05 , triggered on 2017-09-10 for hypotension with BP 67/38 and transferred back to CCU on 2017-09-10 .

text S1 , S2 with Grade II/VI SEM . +S3 .

text Soft , NT/ND with normoactive BS .

text He had 4 surgeries ( 3 cardiac surgeries ) earlier in the summer , so it is likely that he required copious IVF hydration surrounding these procedures which may have been the cause of his massive fluid overload .

text This acutely worsened on 2017-09-18 , when the patient was found to have mental status changes and an ABG revealed a CO2 of over 100 , likley due to a depressed respiratory drive secondary to sedatives given for agitation on top of a preexisting alkalosis .

text 1. Docusate Sodium 100 mg Capsule Sig : One ( 1 ) Capsule PO TID

In [8]:
import pandas as pd
import json
def sleep_annotation():
    sleep_temp = "/home/yuj49/ConText/ConText_LLM/sleep_annotation/results_yuelyu.csv"
    data = pd.read_csv(sleep_temp)
    non_na_data = data[data['labels_z'].notna()]


    
    
    non_na_data = non_na_data[
        ~non_na_data['labels_z'].str.contains(' ') |
        non_na_data['labels_z'].str.contains('fam') |
        non_na_data['labels_z'].str.contains('his') |
    non_na_data['labels_z'].str.contains('pos')|
    non_na_data['labels_z'].str.contains('neg')|
    non_na_data['labels_z'].str.contains('hyp')
    ]
    # 拆分含多个标签的字段
    non_na_data['labels_z'] = non_na_data['labels_z'].str.split()

    # 将列表中的每个标签转换为单独的行
    filtered_data = non_na_data.explode('labels_z')
    print(filtered_data.head())
    # filtered_data = data[data['labels_x'].astype(str).notna() & (~data['labels_x'].astype(str).str.contains(' '))]

    result = filtered_data[['file', 'concept', 'labels_z', 'text']]
    label_mapping = {
        'hyp': 'hypothetical',
        'his': 'historical',
        'neg': 'negated',
        'pos': 'possible',
        'fam': 'family'
        }

    result['labels_z'] = result['labels_z'].map(label_mapping)
    data_list = result.to_dict('records')
    json_list = []

    for item in data_list:
        json_item = {
            "entity": item['concept'],
            "label": item['labels_z'],
            "text": item['text'],
            "file_name": item['file']
        }
        # res = chatgpt_tookit(item['concept'],item['text']) # 287 time call api
        # print(res)
        # json_item["res"] = res
        json_list.append(json_item)
    print(len(json_list))
    with open('/home/yuj49/ConText/ConText_LLM/sleep.json', 'w', encoding='utf-8') as f:
        json.dump(json_list, f, ensure_ascii=False, indent=4)

sleep_annotation() 

   Unnamed: 0                                           file  sentence_id  \
0           0  9554377322_08-23-2019_OFFICE-VISIT_4470169155            7   
1           1  9554681758_06-09-2020_OFFICE-VISIT_4364912250            7   
3           3  9554821474_12-19-2018_OFFICE-VISIT_4448289698           13   
4           4  9554821474_12-19-2018_OFFICE-VISIT_4448289698           44   
5           5  9554821474_12-19-2018_OFFICE-VISIT_4448289698           44   

       concept labels_x labels_y labels_z  doc_index  \
0  sleep_apnea      NaN      hyp      hyp         67   
1  sleep_apnea      his      his      his        181   
3      snoring      his      NaN      his        249   
4  sleep_apnea      pos      pos      pos        280   
5      snoring      his      pos      his        280   

                                                text  
0  allergies   allergen reactions     protonix [p...  
1  past medical history:   past medical history: ...  
3  still snores and had limb moveme

/tmp/ipykernel_135996/607779435.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['labels_z'] = result['labels_z'].map(label_mapping)


In [2]:
# rs= "rs"
# prompt = (
#         f"Please assess the following statement and determine if it's true or false: '{rs}'"
#         " Your response should only contain 'True' or 'False'."
#     )
# prompt

"Please assess the following statement and determine if it's true or false: 'rs' Your response should only contain 'True' or 'False'."

# i2b2 sft

In [35]:
import json
import random
from sklearn.model_selection import train_test_split


file_name = "/home/yuj49/ConText/ConText_LLM/i2b2.json"

with open(file_name,"r") as f:
    data = json.load(f)
print(len(data))
def create_prompt(entity, label, text, is_true):
    prompt = {
                "positive": f"Determine if the condition '{entity}' is currently affecting the patient as mentioned in the text: '{text}'.",
                "negated": f"Identify if the condition '{entity}' is not present in the patient as mentioned in the text: '{text}'.",
                "possible": f"Assess if there is a possibility of the condition '{entity}' affecting the patient as mentioned in the text: '{text}'.",
                "family": f"Verify if the condition '{entity}' is related to the patient's family member as mentioned in the text: '{text}'.",
                "historical": f"Check if the condition '{entity}' is part of the patient's historical medical records as mentioned in the text: '{text}'.",
                "hypothetical": f"Consider if the condition '{entity}' is mentioned hypothetically as it might affect the patient as mentioned in the text: '{text}'."
    }
    result = prompt[label]
    target = "True" if is_true else "False"
    prompt = (
        f"Please assess the following statement and determine if it's true or false: '{result}'"
        " Your response should only contain 'True' or 'False'."
        f"Answer: '{target}'."
    )
    return prompt


# Possible labels for generating false samples
possible_labels = ['positive', 'negated', 'possible', 'family', 'historical', 'hypothetical']
negative_sample_ratio = 0.25
number_of_negative_samples = int(len(data) * negative_sample_ratio)
# Randomly select items to generate negative samples
negative_sample_indices = random.sample(range(len(data)), number_of_negative_samples)


def generate_false_label(true_label, possible_labels):
    false_labels = [label for label in possible_labels if label != true_label]
    return random.choice(false_labels)

prompts = []
for i, item in enumerate(data):
    prompt_text = create_prompt(item["entity"], item["label"], item["text"], True)
    prompt = {'id': i, 'text': prompt_text,"label":item["label"]}
    prompts.append(prompt)
    
    if i in negative_sample_indices:
        false_label = generate_false_label(item["label"], possible_labels)
        false_prompt_text = create_prompt(item["entity"], false_label, item["text"], False)
        false_prompt = {'id': i+len(data), 'text': false_prompt_text, "label":false_label}
        prompts.append(false_prompt)


train_prompts, test_prompts = train_test_split(prompts, test_size=0.2, random_state=42)

# Second split to separate validation from the temporary set
train_prompts, val_prompts = train_test_split(train_prompts, test_size=0.125, random_state=42)  # 0.125 x 0.8 = 0.1

new_train = "/home/yuj49/ConText/ConText_LLM/sft_data/i2b2/train.json"
new_val = "/home/yuj49/ConText/ConText_LLM/sft_data/i2b2/val.json"
new_test = "/home/yuj49/ConText/ConText_LLM/sft_data/i2b2/test.json"
with open(new_train, "w") as f:
    json.dump(train_prompts, f, indent=4)

with open(new_val, "w") as f:
    json.dump(val_prompts, f, indent=4)

with open(new_test, "w") as f:
    json.dump(test_prompts, f, indent=4)



4187


In [33]:
len(train_prompts)

3662

# sleep sft

In [34]:
import json
import random
from sklearn.model_selection import train_test_split

file_name = "/home/yuj49/ConText/ConText_LLM/sleep.json"

with open(file_name,"r") as f:
    data = json.load(f)
print(len(data))
def create_prompt(entity, label, text, is_true):
    prompt = {
                "positive": f"Determine if the condition '{entity}' is currently affecting the patient as mentioned in the text: '{text}'.",
                "negated": f"Identify if the condition '{entity}' is not present in the patient as mentioned in the text: '{text}'.",
                "possible": f"Assess if there is a possibility of the condition '{entity}' affecting the patient as mentioned in the text: '{text}'.",
                "family": f"Verify if the condition '{entity}' is related to the patient's family member as mentioned in the text: '{text}'.",
                "historical": f"Check if the condition '{entity}' is part of the patient's historical medical records as mentioned in the text: '{text}'.",
                "hypothetical": f"Consider if the condition '{entity}' is mentioned hypothetically as it might affect the patient as mentioned in the text: '{text}'."
    }
    result = prompt[label]
    target = "True" if is_true else "False"
    prompt = (
        f"Please assess the following statement and determine if it's true or false: '{result}'"
        " Your response should only contain 'True' or 'False'."
        f"Answer: '{target}'."
    )
    return prompt


# Possible labels for generating false samples
possible_labels = ['positive', 'negated', 'possible', 'family', 'historical', 'hypothetical']
negative_sample_ratio = 0.25
number_of_negative_samples = int(len(data) * negative_sample_ratio)
# Randomly select items to generate negative samples
negative_sample_indices = random.sample(range(len(data)), number_of_negative_samples)


def generate_false_label(true_label, possible_labels):
    false_labels = [label for label in possible_labels if label != true_label]
    return random.choice(false_labels)

prompts = []
for i, item in enumerate(data):
    prompt_text = create_prompt(item["entity"], item["label"], item["text"], True)
    prompt = {'id': i, 'text': prompt_text,"label": item["label"]}
    prompts.append(prompt)
    
    if i in negative_sample_indices:
        false_label = generate_false_label(item["label"], possible_labels)
        false_prompt_text = create_prompt(item["entity"], false_label, item["text"], False)
        false_prompt = {'id': i+len(data), 'text': false_prompt_text,"label": false_label}
        prompts.append(false_prompt)


train_temp, test_prompts = train_test_split(prompts, test_size=0.2, random_state=42)

# Second split to separate validation from the temporary set
train_prompts, val_prompts = train_test_split(train_temp, test_size=0.125, random_state=42)  # 0.125 x 0.8 = 0.1

new_train = "/home/yuj49/ConText/ConText_LLM/sft_data/sleep/train.json"
new_val = "/home/yuj49/ConText/ConText_LLM/sft_data/sleep/val.json"
new_test = "/home/yuj49/ConText/ConText_LLM/sft_data/sleep/test.json"
with open(new_train, "w") as f:
    json.dump(train_prompts, f, indent=4)

with open(new_val, "w") as f:
    json.dump(val_prompts, f, indent=4)

with open(new_test, "w") as f:
    json.dump(test_prompts, f, indent=4)


111


In [29]:
len(train_prompts)

96

In [30]:
len(val_prompts)

14

In [31]:
len(test_prompts)

28

# sleep annotated

In [1]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# 初始化BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text):
    """
    获取文本的BERT嵌入。
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def calculate_similarity(sentence, word_embedding):
    """
    计算句子和特定词之间的相似度。
    """
    sentence_embedding = get_embedding(sentence)
    return torch.nn.functional.cosine_similarity(sentence_embedding, word_embedding).numpy()

# 获取“sleep”一词的嵌入
sleep_embedding = get_embedding("sleep")

# 读取和处理文件
def read_and_process_file(file_path):
    sleep_related_sentences = []

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    sentences = re.split(r'[.!?]', content)

    for i, sentence in enumerate(sentences):
        similarity = calculate_similarity(sentence, sleep_embedding)
        if similarity > 0.5:  # 设置一个阈值
            sleep_related_sentences.append((sentence.strip(), i + 1, similarity))

    return sleep_related_sentences


/home/yuj49/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import re
import csv
import os
from tqdm import tqdm

def read_and_split_sentences(file_path):
    """
    Read the content of a file, split it into sentences, and check if each sentence is related to sleep.
    """
    sleep_related_sentences = []

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    sentences = re.split(r'[.!?]', content)

    for i, sentence in enumerate(sentences):
        if 'sleep' in sentence.lower():
            sleep_related_sentences.append((sentence.strip(), i + 1))

    return sleep_related_sentences

def write_to_csv(file_name, sleep_related_sentences, output_csv):
    """
    Write the sleep-related sentences to a CSV file.
    """
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['File Name', 'Sleep-Related Sentence', 'Sentence Number'])

        for sentence, number in sleep_related_sentences:
            csvwriter.writerow([file_name, sentence, number])

# Example usage
path_file = "/home/yuj49/ConText/ConText_LLM/sleep_data"
for i in os.listdir(path_file):
    file_path = os.path.join(path_file, i)
all_sen = []
output_csv = 'output.csv'  # Replace with the path to your desired output CSV file
file_path = []
for i in os.listdir(path_file):
    file_path = os.path.join(path_file, i) # Replace with the path to your file
    
    sleep_related_sentences = read_and_split_sentences(file_path)
    file_path.extend([len(sleep_related_sentences)*i ])
    all_sen.extend(sleep_related_sentences)

write_to_csv(file_path, sleep_related_sentences, output_csv)


In [12]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import re
import csv
from tqdm import tqdm

# 初始化BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text):
    """获取文本的BERT嵌入。"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def calculate_similarity(sentence, word_embedding):
    """计算句子和特定词之间的相似度。"""
    sentence_embedding = get_embedding(sentence)
    return torch.nn.functional.cosine_similarity(sentence_embedding, word_embedding).numpy()

# 获取“sleep”一词的嵌入
sleep_embedding = get_embedding("sleep")

def read_and_process_file(file_path):
    """读取和处理单个文件，寻找与'sleep'相关的句子。"""
    sleep_related_sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    sentences = re.split(r'[.!?]', content)
    for i, sentence in enumerate(sentences):
        similarity = calculate_similarity(sentence, sleep_embedding)
        if similarity > 0.3 and len(sentence.split(" ")) >5:  # 设置一个阈值
            sleep_related_sentences.append((file_path, sentence.strip(), i + 1, similarity))

    return sleep_related_sentences

def process_files(file_paths, output_csv):
    """处理多个文件并将结果写入CSV文件。"""
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['File Name', 'Sleep-Related Sentence', 'Sentence Number', 'Similarity'])

        for file_path in tqdm(file_paths):
            sleep_related_sentences = read_and_process_file(file_path)
            for row in sleep_related_sentences:
                csvwriter.writerow(row)

# 示例使用
# file_paths = ['file1.txt', 'file2.txt', 'file3.txt']  # 替换为您的文件路径
output_csv = 'sleep_related_sentences.csv'
path_file = "/home/yuj49/ConText/ConText_LLM/sleep_data"
file_path =  []
for i in os.listdir(path_file):
    file_path.append(os.path.join(path_file, i))
process_files(file_path, output_csv)


100%|███████████████████████████████████████████████████| 53/53 [01:52<00:00,  2.13s/it]


In [18]:
import pandas as pd
# 读取CSV文件
df = pd.read_csv("sleep_related_sentences.csv")
df['Similarity'] = pd.to_numeric(df['Similarity'], errors='coerce')

# 筛选出相似度大于0.4的行
df_ = df.loc[df['Similarity'] > 0.4]

In [19]:
len(df_)

0

In [20]:
df['Similarity']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2115   NaN
2116   NaN
2117   NaN
2118   NaN
2119   NaN
Name: Similarity, Length: 2120, dtype: float64

In [3]:
import csv

def extract_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    records = []
    current_record = {}
    response_flag = False  # 用于标记是否正在读取RESPONSE部分
    response_lines = []

    for line in lines:
        line = line.strip()  # 移除行首尾的空白字符
        if line.startswith('RECORD TEXT:'):
            if current_record:
                current_record['response'] = ' '.join(response_lines)
                records.append(current_record)
            current_record = {'record_text': line[len('RECORD TEXT:'):].strip()}
            response_flag = False
            response_lines = []
        elif line.startswith('RESPONSE:'):
            response_flag = True
            response_lines.append(line[len('RESPONSE:'):].strip())
        elif response_flag:
            response_lines.append(line)

    if current_record:
        current_record['response'] = ' '.join(response_lines)
        records.append(current_record)

    return records

def save_to_csv(records, output_file_path):
    with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['record_text', 'response'])
        writer.writeheader()
        for record in records:
            writer.writerow(record)


# 使用示例
file_path = '/home/yuj49/ConText/ConText_LLM/annotation_results_context.txt' # 将 'your_data_file.txt' 替换为你的数据文件名
output_file_path = 'sleep_256.csv' # 输出文件的路径和名称
records = extract_data(file_path)
save_to_csv(records, output_file_path)
